# Python und MongoDB

Dieses Beispiel setzt einen laufenden MongoDB-Server mit aktivierter Authentifizierung auf Localhost voraus. Ebenso ist ein User (mit Namen und Passwort "pkmlp") für die Datenbank "webshop" mit der Collection "artikel" definiert. Dieses Programm simuliert die Arbeit (CRUD von Dokumenten in der Sammlung) mit einer vorhandenen Datenbank. Zur Kontrolle und zur Verfolgung der einzelnen Schritte dieses Beispiel-Programmes ist weiter ein MongoDB-GUI (z.B. MongoClient, MongoBooster, Studio 3T, Robo 3T, MongoCompass, etc.) sehr empfohlen. 

Kontrolliere in einem der MongoDB-GUIs oder im Mongo-Terminal ob die Datenbank "webshop" mit der Dokumenten-Sammlung (Collection) "artikel" und der entsprechende User "pkmlp" mit den notwendigen Rechten auf der Datenbank vorhanden ist. 

Falls nicht,als mongoAdmin einloggen und die Datenbank "webshop" erstellen und den User "pkmlp" für die Datenbank aufnehmen (als dbOwner).  

## Erstellen der Verbindung zum Datenbank Server 

Um aus Python auf eine NoSQL-Datenbank (hier MongoDB) zugreifen zu können, muss die entsprechende Bibliothek eingebunden werden. 

In [ ]:
import pymongo

Definition der Datenbank-Parameter als Konstanten

In [ ]:
DATABASE_HOST      =  "localhost"
DATABASE_PORT      =  "27017"
DATABASE_USER      =  "pkmlp"
DATABASE_PASSWORT  =  "pkmlp"
DATABASE_NAME      =  "webshop"
COLLECTION_NAME    =  "artikel"

Als nächstes muss eine Verbindung zur Datenbank (mit gültigen Credentials) hergestellt werden.  

In [ ]:
try:
    dbVerbindung = pymongo.MongoClient('mongodb://'+DATABASE_USER+':'+DATABASE_PASSWORT+'@'+
                                       DATABASE_HOST+':'+DATABASE_PORT+'/'+DATABASE_NAME)
    print("\nVerbindung zu MongoDB erstellt\n")
except pymongo.errors.ConnectionFailure as VerbindungsFehler:
    print("\nKeine Verbindung zu MongoDB: ", VerbindungsFehler, " - Programmabbruch\n")

## Auswählen der Datenbank und der Sammlung

Verbindung konnte erstellt werden, nun die Datenbank ...

In [ ]:
db = dbVerbindung[DATABASE_NAME]

... und die Collection in der Datenbank wählen

In [ ]:
sammlung = db[COLLECTION_NAME]

### Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal ob die Collection wirklich leer ist.

Die Collection muss nicht zwingend leer sein. Ist sie es nicht, so muss der Anfangs-Datenbestand berücksichtigt werden. Das heisst, die nachfolgend eingefügten (Create), gelesenen (Read), mutierten (Update) und gelöschten (Delete) Dokumente sind immer auch mit den allenfalls bereits vorhandenen Dokumenten "zu sehen". Dieses Beispiel ist besser zu Interpretieren, wenn die Sammlung vor Beginn leer ist.

Definieren der einzufügenden Artikel. Beachte: die Artikel müssen nicht eine einheitliche Struktur (gleiche Attribute) haben. 

In [ ]:
dokument_1 = { 
    "artikel_nr" : 1, 
    "artikel_bezeichnung" : "Artikel-1", 
    "artikel_beschreibung" : "Artikel 1 in den Kategorien 1 und 2 mit 4 Attributen (grösse, farbe, jahrgang, getriebe) und 3 Stichworten (cool, mega, bombastisch)", 
    "artikel_preis" : 100.0, 
    "artikel_attribute" : {
        "grösse" : 185, 
        "farbe" : "Rot", 
        "jahrgang" : 1960, 
        "getriebe" : "Automatik"
    }, 
    "artikel_stichworte" : [
        "cool", 
        "mega", 
        "bombastisch"
    ]
}

dokument_2 = { 
    "artikel_nr" : 2, 
    "artikel_bezeichnung" : "Artikel-2", 
    "artikel_beschreibung" : "Artikel 2 in den Kategorien 1 und 3 mit 3 Attributen (grösse, farbe, gewicht) und 2 Stichworten (cool, lässig)", 
    "artikel_preis" : 200.0, 
    "artikel_attribute" : {
        "grösse" : 145, 
        "farbe" : "Blau", 
        "gewicht" : 15.5
    }, 
    "artikel_stichworte" : [
        "cool", 
        "lässig"
    ]
}

## Einfügen von Dokumenten in die Sammlung (Collection) der Datenbank

Einfügen der einzeln definierten Dokumente in die Datenbank/Collection

In [ ]:
insertedKey = sammlung.insert_one(dokument_1)
print("\nPrimekey des eingefügten Artikels: ", insertedKey.inserted_id)

insertedKey = sammlung.insert_one(dokument_2)
print("Primekey des eingefügten Artikels: ", insertedKey.inserted_id)


### Kontrolliere in einem MongoDB Gui-Client oder im mongo Terminal die Collection.

War die Sammlung zu Beginn leer, so dürfen jetzt nur die 2 obigen Dokumente darin sein. War die Sammlung zu Beginn nicht leer, so sind jetzt zusätzlich zu den bereits vorhandenen Dokumenten die obigen 2 in der Sammlung, da wir ja den PrimeKey (\_id) in unseren Dokumenten nicht definiert haben, wird jedes Dokument eingefügt und MongoDB erstellt automatisch den Primekey.

## Suchen und Zählen von Dokumenten in der Sammlung

In [ ]:
anzDokumente = sammlung.find().count()
print("Der WebShop enthält",anzDokumente, "Artikel\n")

## Lesen und Ausgeben von Dokumenten in der Sammlung 

Lesen aller Dokumente aus der Datenbank
Hinweis: Die Reihenfolge der einzelnen Attribute in einer Zeile sind nicht definiert.
Darum müssen Attribute gezielt mit dem Key aus dem Cursor gelesen werden.

In [ ]:
print("Lesen aller Dokumente nach Einfuegen")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)

In [ ]:
print("\nLesen aller Dokumente aber nur Attributswerte ausgeben")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument.get("artikel_nr"), 
          dokument.get("artikel_bezeichnung"), 
          dokument.get("artikel_beschreibung"),
          dokument.get("artikel_preis"))

In [ ]:
print("\nLesen aller Dokumente absteigend sortiert nach artikel_nr")
dokumentCursor = sammlung.find().sort("artikel_nr",-1)
for dokument in dokumentCursor:
    print(dokument.get("artikel_nr"), 
          dokument.get("artikel_bezeichnung"), 
          dokument.get("artikel_beschreibung"),
          dokument.get("artikel_preis"))

## Lesen/Suchen und Ausgeben von Artikeln 

### Lesen von Dokumenten mit Suchkriterium 'artikel_nr = 2'

In [ ]:
dokumentCursor = sammlung.find({"artikel_nr":2})
for dokument in dokumentCursor:
    print(dokument)

In [ ]:
suchKriterium = {"artikel_nr":2}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

### Lesen von Dokumenten mit Suchkriterium 'artikel_nr ungleich 2'

In [ ]:
dokumentCursor = sammlung.find({"artikel_nr":{"$ne":2}})
for dokument in dokumentCursor:
    print(dokument)

In [ ]:
suchKriterium = {"artikel_nr":{"$ne":2}}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

### Lesen von Dokumenten mit Suchkriterien 'artikel_preis kleiner 150'

In [ ]:
dokumentCursor = sammlung.find({"artikel_preis":{"$lt":150}})
for dokument in dokumentCursor:
    print(dokument)

In [ ]:
suchKriterium = {"artikel_preis":{"$lt":150}}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

### Lesen von Dokumenten mit "Suchstring" --> Artikel-Beschreibung enthält das Work 'lässig'

Hinweis: In diesem Beispiel soll nur im Attribut *artikel_beschreibung*  gesucht werden. Hier wird noch nicht geprüft, ob das Wort 'lässig' in den Stichworten vorkommt. Das kommt im nächsten Kapitel.

In [ ]:
dokumentCursor = sammlung.find({"artikel_beschreibung":{"$regex": u"lässig"}})
for dokument in dokumentCursor:
    print(dokument)

In [ ]:
suchKriterium = {"artikel_beschreibung":{"$regex": u"lässig"}}
dokumentCursor = sammlung.find(suchKriterium)
for dokument in dokumentCursor:
    print(dokument)

Variere die Stichworte um zu sehen, dass es wirklich funktioniert. Erwartete Resultate:
- lässig sollte 1 Artikel finden
- cool sollte 2 Artikel finden,

### Lesen von Dokumenten mit "Suchstring" --> Artikel-Stichworte enthält 'cool'

In [ ]:
dokumentCursor = sammlung.find({"artikel_stichworte":{"$in":["cool"]}})
for dokument in dokumentCursor:
    print(dokument)

Variere die Stichworte um zu sehen, dass es wirklich funktioniert. Erwartete Resultate:
- cool sollte 2 Artikel finden,
- lässig sollte 1 Artikel finden

### Verarbeiten von Dokumenten mit "Suchstring" --> Artikel-Stichworte enthält 'cool'

In diesem Beispiel soll der gefunden Artikel mit allen Details ("Attribut für Attribut", "Stichwort für Stichwort") verabreitet (ausgegeben) werden.

In [ ]:
dokumentCursor = sammlung.find({"artikel_stichworte":{"$in":["cool"]}})
for dokument in dokumentCursor:
    print()
    print("Artikel-Nr:           ", dokument.get("artikel_nr")) 
    print("Artikel-Bezeichnung:  ", dokument.get("artikel_bezeichnung"))
    print("Artikel-Beschreibung: ", dokument.get("artikel_beschreibung"))
    print("Artikel-Preis:        ", dokument.get("artikel_preis"))
    print()
    print("Attribute des Artikels:")
    attribute = dokument.get("artikel_attribute")
    for attribut in attribute.items():
        print("   - ",attribut[0], ":", attribut[1])
    print()
    print("Stichworte zum Artikel:")
    for stichwort in dokument.get("artikel_stichworte"):
        print("   - ", stichwort)
    print()
    print()

Wenn es nur um einen schönen (besser lesbaren) Ausdruck geht:

In [ ]:
import pprint

dokumentCursor = sammlung.find({"artikel_stichworte":{"$in":["cool"]}})
for dokument in dokumentCursor:
    print()
    print(pprint.pformat(dokument))
    print()
    print()

PPRINT sortiert die Ausgabe nach den Schlüsseln (Keys). Etwas umständlicher, dafür ohne Sortierung, geht es mit dem Modul JSON. Umständlicher ist es nur wegen MongoDB (siehe Erklärung nach dem Beispiel).

In [ ]:
import json
from bson import json_util

dokumentCursor = sammlung.find({"artikel_stichworte":{"$in":["lässig"]}})
for dokument in dokumentCursor:
    print()
    print(json.dumps(json.loads(json_util.dumps(dokument)), ensure_ascii=False, indent=4))
    print()
    print()

MongoDB speichert die Daten im BSON Format. Damit kommt das Modul JSON nicht zurecht. Darum muss das Dokument mit json_util.dumps aus der Bibliothek BSON in "normales JSON" gewandelt werden. Nach der Umwandlung ist dies jedoch ein String. Darum muss dieser dann mit json.loads als JSON erkannt/gelesen/umgewandelt werden, damit dieser dann mit json.dumps fomratiert ausgegeben werden kann. Als Parameter für die Formatierung habe ich hier den Einzug (indent) angegeben und mit ensure_ascii werden auch die Deutschen Umlaute korrekt ausgegeben. 

## Aufgabe

Erweitere dieses Notebook mit Beispielen in denen Du z.B. bestimmte Stichworte und eine Preisobergrenze definierst (halt so, wie Du in Deinem Internet-Shops etwas suchen würdest). 

## Löschen der Dokumenten in der Sammlung 

Löschen der Artikel im WebShop (aufräumen)

In [ ]:
print("\nLöschen aller Dokumente")
anzDelete = sammlung.delete_many({})
print("Anzahl gelöschter Dokumente: ", anzDelete.deleted_count)

Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal ob die Collection wirklich leer ist.

Verbindung zur Datenbank beenden

In [ ]:
dbVerbindung.close()

That's all Folks